NOMBRE: LRCQ                                                    CARRERA:CICO

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import time
from stable_baselines3 import PPO

class RoboticArmEnv(gym.Env):
    def _init_(self, render_mode=None):
        super()._init_()
        self.render_mode = render_mode
        self.observation_space = spaces.Box(low=-1.0, high=1.0, shape=(6,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(3,), dtype=np.float32)
        self.physics_client = None

    def reset(self, seed=None, options=None):
        if self.physics_client is not None:
            p.disconnect(self.physics_client)
        self.physics_client = p.connect(p.GUI if self.render_mode == "human" else p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.resetSimulation()
        p.setGravity(0, 0, -9.8)
        p.loadURDF("plane.urdf")
        self.arm_id = p.loadURDF("kuka_iiwa/model.urdf", useFixedBase=True)
        self.cup_id = p.loadURDF("cube_small.urdf", [np.random.uniform(0.3, 0.7), 0, 0.05])
        return self._get_obs(), {}

    def _get_obs(self):
        joint_states = p.getJointStates(self.arm_id, [0, 1, 2])
        joints = np.array([s[0] for s in joint_states])
        cup_pos, _ = p.getBasePositionAndOrientation(self.cup_id)
        return np.concatenate((joints, np.array(cup_pos[:3])), axis=0)

    def step(self, action):
        for i in range(3):
            p.setJointMotorControl2(self.arm_id, i, p.POSITION_CONTROL, targetPosition=action[i])
        p.stepSimulation()
        time.sleep(1/60.0)
        obs = self._get_obs()
        reward = -0.01
        done = False
        cup_pos, _ = p.getBasePositionAndOrientation(self.cup_id)
        if cup_pos[2] < 0.02:
            reward = -1.0
            done = True
        elif abs(cup_pos[0] - 0.5) < 0.05:
            reward = 1.0
            done = True
        return obs, reward, done, False, {}

    def render(self):
        pass

    def close(self):
        if self.physics_client:
            p.disconnect(self.physics_client)
            self.physics_client = None


# ENTRENAMIENTO
env = RoboticArmEnv(render_mode=None)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("ppo_robot_arm")
env.close()

# EVALUACIÓN
env = RoboticArmEnv(render_mode="human")
model = PPO.load("ppo_robot_arm")
obs, _ = env.reset()
for _ in range(200):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    if done:
        print(f"Episode finished with reward: {reward}")
        obs, _ = env.reset()
env.close()